In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import os
import shutil
from pathlib import Path

print('TensorFlow version:', tf.__version__)

## Step 1: Merge Datasets
Combine the downloaded 60GB dataset with your Spicy_Ai images. We'll create a binary classifier:
- **NSFW class**: porn + hentai + sexy categories + your Spicy_Ai images
- **Safe class**: neutral + drawings categories + your non-sensitive images

## Step 0: Download 60GB NSFW Dataset (NSFWJS Training Data)
This will clone the nsfw_data_scraper repo and automatically download ~60GB of labeled images from Reddit and other sources. **This will take several hours** - best to run overnight!

In [ ]:
# Clone the NSFW data scraper repository
import subprocess
import sys

# Install required packages
!pip install wget

# Clone the repo if not already present
scraper_dir = r'nsfw_data_scraper'
if not os.path.exists(scraper_dir):
    print('Cloning NSFW data scraper repository...')
    !git clone https://github.com/alex000kim/nsfw_data_scraper.git
    print('✓ Repository cloned!')
else:
    print('✓ Repository already exists!')

In [ ]:
# Download Ripme tool (required for scraping)
ripme_path = r'nsfw_data_scraper\ripme.jar'
if not os.path.exists(ripme_path):
    print('Downloading Ripme tool...')
    import urllib.request
    ripme_url = 'https://github.com/RipMeApp/ripme/releases/latest/download/ripme.jar'
    urllib.request.urlretrieve(ripme_url, ripme_path)
    print('✓ Ripme downloaded!')
else:
    print('✓ Ripme already downloaded!')

In [ ]:
# Run the automated download scripts
# WARNING: This downloads ~60GB of data and will take HOURS (3-6 hours depending on internet speed)
# It scrapes images from Reddit and categorizes them into: porn, hentai, sexy, neutral, drawings

print('Starting automated download...')
print('This will download ~60GB of NSFW training data from multiple sources')
print('Expected time: 3-6 hours')
print('')

os.chdir('nsfw_data_scraper')

# Step 1: Get image URLs from source lists (fast - a few minutes)
print('Step 1/3: Getting image URLs...')
!bash scripts/1_get_urls.sh

# Step 2: Download actual images from URLs (SLOW - several hours)
print('Step 2/3: Downloading images (this takes the longest)...')
!bash scripts/2_download_from_urls.sh

# Step 3: Organize into train directory
print('Step 3/3: Organizing images...')
!bash scripts/5_create_train.sh

os.chdir('..')
print('')
print('✓ Dataset download complete!')
print('Images saved to: nsfw_data_scraper/data/train/')
print('')
print('Categories:')
print('  - porn/')
print('  - hentai/')
print('  - sexy/')
print('  - neutral/')
print('  - drawings/')

### Windows Alternative (if bash scripts don't work)
If the bash scripts fail, use this Python-based downloader instead:

In [ ]:
# Windows-compatible Python downloader
# This manually processes the URL lists and downloads images

import urllib.request
import time
from pathlib import Path

def download_from_url_file(url_file, output_dir):
    """Download images from a text file containing URLs"""
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    if not os.path.exists(url_file):
        print(f'⚠ File not found: {url_file}')
        return 0
    
    with open(url_file, 'r', encoding='utf-8', errors='ignore') as f:
        urls = [line.strip() for line in f if line.strip()]
    
    print(f'Found {len(urls)} URLs in {os.path.basename(url_file)}')
    downloaded = 0
    
    for i, url in enumerate(urls):
        if i % 100 == 0:
            print(f'  Progress: {i}/{len(urls)} ({downloaded} successful)')
        
        try:
            # Extract filename from URL
            filename = url.split('/')[-1]
            if not any(filename.endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
                filename += '.jpg'
            
            output_path = os.path.join(output_dir, filename)
            
            # Download with timeout
            urllib.request.urlretrieve(url, output_path)
            downloaded += 1
            time.sleep(0.1)  # Be nice to servers
            
        except Exception as e:
            pass  # Skip failed downloads
    
    print(f'✓ Downloaded {downloaded}/{len(urls)} images')
    return downloaded

# Process each category
scraper_dir = r'nsfw_data_scraper'
raw_data_dir = os.path.join(scraper_dir, 'raw_data')
output_dir = os.path.join(scraper_dir, 'data', 'train')

categories = {
    'porn_urls.txt': 'porn',
    'hentai_urls.txt': 'hentai',
    'sexy_urls.txt': 'sexy',
    'neutral_urls.txt': 'neutral',
    'drawings_urls.txt': 'drawings'
}

print('Starting Windows-compatible download...')
print('This will take several hours for 60GB of data\n')

total_downloaded = 0
for url_file, category in categories.items():
    print(f'\n{"="*60}')
    print(f'Downloading {category.upper()} category')
    print(f'{"="*60}')
    
    url_path = os.path.join(raw_data_dir, url_file)
    output_path = os.path.join(output_dir, category)
    
    count = download_from_url_file(url_path, output_path)
    total_downloaded += count

print(f'\n{"="*60}')
print(f'DOWNLOAD COMPLETE')
print(f'Total images downloaded: {total_downloaded:,}')
print(f'{"="*60}')

In [ ]:
# Paths to datasets
downloaded_data = r'nsfw_data_scraper\data\train'
spicy_source = r'C:\Users\Shado\Downloads\Spicy_Ai\Spicy_Ai_Training\samples'
safe_source = r'dataset\training\nonsensitive'
output_base = r'nsfw_dataset'

# Create directory structure
dirs = [
    f'{output_base}/training/nsfw',
    f'{output_base}/training/safe',
    f'{output_base}/validation/nsfw',
    f'{output_base}/validation/safe'
]

for d in dirs:
    Path(d).mkdir(parents=True, exist_ok=True)
    
print('✓ Dataset directories created!')
print('')
print('Merging strategy:')
print('  NSFW = porn + hentai + sexy + Spicy_Ai')
print('  Safe = neutral + drawings + your non-sensitive images')

In [ ]:
# Collect all NSFW images from downloaded dataset + your Spicy_Ai
import glob

print('Collecting NSFW images...')

nsfw_images = []

# Add porn category
porn_path = os.path.join(downloaded_data, 'porn')
if os.path.exists(porn_path):
    porn_imgs = glob.glob(os.path.join(porn_path, '*.*'))
    nsfw_images.extend(porn_imgs)
    print(f'  ✓ Porn: {len(porn_imgs)} images')

# Add hentai category
hentai_path = os.path.join(downloaded_data, 'hentai')
if os.path.exists(hentai_path):
    hentai_imgs = glob.glob(os.path.join(hentai_path, '*.*'))
    nsfw_images.extend(hentai_imgs)
    print(f'  ✓ Hentai: {len(hentai_imgs)} images')

# Add sexy category
sexy_path = os.path.join(downloaded_data, 'sexy')
if os.path.exists(sexy_path):
    sexy_imgs = glob.glob(os.path.join(sexy_path, '*.*'))
    nsfw_images.extend(sexy_imgs)
    print(f'  ✓ Sexy: {len(sexy_imgs)} images')

# Add your Spicy_Ai images
if os.path.exists(spicy_source):
    spicy_imgs = glob.glob(os.path.join(spicy_source, '*.*'))
    nsfw_images.extend(spicy_imgs)
    print(f'  ✓ Spicy_Ai: {len(spicy_imgs)} images')

print(f'\nTotal NSFW images: {len(nsfw_images)}')

# Shuffle and split 80/20
np.random.shuffle(nsfw_images)
split_idx = int(len(nsfw_images) * 0.8)
train_nsfw = nsfw_images[:split_idx]
val_nsfw = nsfw_images[split_idx:]

# Copy training NSFW
print(f'\nCopying {len(train_nsfw)} training images...')
for i, img in enumerate(train_nsfw):
    if i % 1000 == 0:
        print(f'  Progress: {i}/{len(train_nsfw)}')
    try:
        shutil.copy(img, f'{output_base}/training/nsfw/')
    except Exception as e:
        pass  # Skip corrupted images
    
# Copy validation NSFW
print(f'Copying {len(val_nsfw)} validation images...')
for img in val_nsfw:
    try:
        shutil.copy(img, f'{output_base}/validation/nsfw/')
    except Exception as e:
        pass

print(f'\n✓ NSFW images copied: {len(train_nsfw)} training, {len(val_nsfw)} validation')

In [ ]:
# Collect all Safe images from downloaded dataset + your non-sensitive images
print('Collecting Safe images...')

safe_images = []

# Add neutral category
neutral_path = os.path.join(downloaded_data, 'neutral')
if os.path.exists(neutral_path):
    neutral_imgs = glob.glob(os.path.join(neutral_path, '*.*'))
    safe_images.extend(neutral_imgs)
    print(f'  ✓ Neutral: {len(neutral_imgs)} images')

# Add drawings category (SFW anime/drawings)
drawings_path = os.path.join(downloaded_data, 'drawings')
if os.path.exists(drawings_path):
    drawing_imgs = glob.glob(os.path.join(drawings_path, '*.*'))
    safe_images.extend(drawing_imgs)
    print(f'  ✓ Drawings: {len(drawing_imgs)} images')

# Add your non-sensitive images
if os.path.exists(safe_source):
    your_safe_imgs = glob.glob(os.path.join(safe_source, '*.*'))
    safe_images.extend(your_safe_imgs)
    print(f'  ✓ Your non-sensitive: {len(your_safe_imgs)} images')

print(f'\nTotal Safe images: {len(safe_images)}')

# Balance dataset (match NSFW count or use all)
if len(safe_images) > len(nsfw_images):
    print(f'Balancing dataset: using {len(nsfw_images)} safe images to match NSFW count')
    safe_images = safe_images[:len(nsfw_images)]

# Shuffle and split 80/20
np.random.shuffle(safe_images)
split_idx = int(len(safe_images) * 0.8)
train_safe = safe_images[:split_idx]
val_safe = safe_images[split_idx:]

# Copy training Safe
print(f'\nCopying {len(train_safe)} training images...')
for i, img in enumerate(train_safe):
    if i % 1000 == 0:
        print(f'  Progress: {i}/{len(train_safe)}')
    try:
        shutil.copy(img, f'{output_base}/training/safe/')
    except Exception as e:
        pass
    
# Copy validation Safe
print(f'Copying {len(val_safe)} validation images...')
for img in val_safe:
    try:
        shutil.copy(img, f'{output_base}/validation/safe/')
    except Exception as e:
        pass

print(f'\n✓ Safe images copied: {len(train_safe)} training, {len(val_safe)} validation')
print('')
print('=' * 60)
print('DATASET SUMMARY')
print('=' * 60)
print(f'Training:   {len(train_nsfw):,} NSFW + {len(train_safe):,} Safe = {len(train_nsfw) + len(train_safe):,} total')
print(f'Validation: {len(val_nsfw):,} NSFW + {len(val_safe):,} Safe = {len(val_nsfw) + len(val_safe):,} total')
print('=' * 60)

## Step 2: Create Data Generators

In [ ]:
# Same preprocessing as your document model
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    f'{output_base}/training',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    f'{output_base}/validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

print('Data generators ready!')

## Step 3: Build Model (Same Architecture as Your Document Model)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

## Step 4: Train the Model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

## Step 5: Save Model

In [ ]:
# Save in H5 format (for compatibility)
model.save('models/nsfw_model.h5')
print('Model saved as models/nsfw_model.h5')

## Step 6: Convert to TensorFlow.js

In [ ]:
# Install tensorflowjs converter if not already installed
!pip install tensorflowjs

In [ ]:
# Convert to TFJS format
!tensorflowjs_converter --input_format=keras models/nsfw_model.h5 webapp/public/models/nsfw_model
print('Model converted for web use!')

## Step 7: Test the Model

In [ ]:
# Test with a few images
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    prediction = model.predict(img_array)[0][0]
    label = 'NSFW' if prediction > 0.5 else 'Safe'
    confidence = prediction if prediction > 0.5 else 1 - prediction
    
    plt.imshow(img)
    plt.title(f'{label} ({confidence*100:.1f}% confidence)')
    plt.axis('off')
    plt.show()
    
    return prediction

# Test with an NSFW image
test_nsfw = val_nsfw[0] if val_nsfw else None
if test_nsfw:
    print('Testing NSFW image:')
    predict_image(test_nsfw)

# Test with a safe image
test_safe = val_safe[0] if val_safe else None
if test_safe:
    print('Testing safe image:')
    predict_image(test_safe)